In [ ]:
# loading the NER model
from library.spacy_ner_model import SpacyNERAnalyzer
model_location = "D:/OneDrive - Sirus NV/ABB/UC0.1/UC1/models/ner/robbert/model-best"
print("Initializing NER analyzer...")
ner_labels = ['CITY', 'DOMAIN', 'HOUSENUMBERS', 'INTERSECTION', 'POSTCODE', 'PROVINCE', 'ROAD', 'STREET']
ner_analyzer = SpacyNERAnalyzer(model_location, labels=ner_labels)





c:\Users\stefa\anaconda3\envs\abb\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\stefa\anaconda3\envs\abb\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'nl_pipeline' (0.0.0) was trained with spaCy v3.6.1 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Initializing NER analyzer...
✅ Loaded model: pipeline (v0.0.0)
Model labels: ('BUILDING', 'CITY', 'DATE', 'DOMAIN', 'HOUSENUMBERS', 'INTERSECTION', 'MONEY', 'ORG', 'PERCENT', 'PERSON', 'POSTCODE', 'PRODUCT', 'PROVINCE', 'ROAD', 'STREET', 'TIME')
Filtering for labels: ['CITY', 'DOMAIN', 'HOUSENUMBERS', 'INTERSECTION', 'POSTCODE', 'PROVINCE', 'ROAD', 'STREET']


In [2]:
# loading the location entity solver
from library.location_solver import LocationEntitiesSolver
solver = LocationEntitiesSolver(ner_analyzer, "Gent")

In [3]:
# Loading the Nominatim geocoder
from library.nominatim_geocoder import NominatimGeocoder
g = NominatimGeocoder(base_url="http://localhost:8080", rate_limit=0.5)


In [5]:
def geocode_detectable(detectable, geocoder, default_city="Gent"):
    """
    Geocode a single detectable entity and return the result.
    
    Args:
        detectable (dict): A detectable entity with name, house_number, city, type etc.
        geocoder (NominatimGeocoder): The geocoder instance to use
        default_city (str): Default city if none specified in the detectable
        
    Returns:
        dict: Geocoding result with success status, display_name, coordinates and LOD data
    """
    # Build the search query based on the detectable type and properties
    name = detectable.get("name", "")
    if not name:
        return {"success": False, "error": "No name in detectable"}
    
    # For houses, include house number in the query
    if detectable.get("type") == "HOUSE" and detectable.get("house_number"):
        query = f"{name} {detectable['house_number']}"
    else:
        query = name
    
    # Use city from detectable or fall back to default
    city = detectable.get("city", default_city)
    
    # Perform the geocoding
    print(f"Geocoding query: '{query}' in city: '{city}'")
    result = geocoder.search(query, city=city)
    
    if result:
        return {
            "success": True,
            "query": query,
            "display_name": result["display_name"],
            "lat": result["lat"],
            "lon": result["lon"],
            "osm_url": result.get("osm_url"),
            "osm_type": result.get("osm_type"),
            "osm_id": result.get("osm_id"),
            "place_id": result.get("place_id"),
            "importance": result.get("importance"),
            "bbox": result.get("bbox"),
            "type": result.get("type"),
            "class": result.get("class"),
            "address": result.get("address"),
            "detectable": detectable
        }
    else:
        return {
            "success": False,
            "query": query,
            "city": city,
            "error": f"No geocoding result found for '{query}' in {city}",
            "detectable": detectable
        }


text = """
2025_BURG_04205 - Burgemeesterbevel houdende flankerende veiligheidsmaatregelen naar aanleiding van de Student Kick-Off 2025 - Goedkeuring
Aan de burgemeester wordt ter goedkeuring voorgelegd het besluit waarmee, net zoals vorige jaren, een aantal flankerende veiligheidsmaatregelen worden ingevoerd naar aanleiding van de Student Kick-Off die op 24 september 2025 doorgaat op het Sint-Pietersplein en het Sint-Amandsplein.

11/09/2025 - Burgemeester - Gent
"""
# Test the new method
detectables, doc = solver.process_text(text)

g = NominatimGeocoder(base_url="http://localhost:8080", rate_limit=0.5)

for entry in detectables:
    result = geocode_detectable(entry, g)
    if result["success"]:
        print( result["display_name"], result["lat"], result["lon"])
        print("   OSM:", result.get("osm_url"))
    else:
        print("error:",result["error"])

c:\Users\stefa\anaconda3\envs\abb\Lib\site-packages\thinc\shims\pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Geocoding query: 'Sint-Pietersplein' in city: 'Gent'
Sint-Pietersplein, Kunstenkwartier, Gent, Oost-Vlaanderen, Vlaanderen, 9000, België / Belgique / Belgien 51.04256585 3.7258780090545707
   OSM: https://www.openstreetmap.org/relation/4621700
Geocoding query: 'Sint-Amandsplein' in city: 'Gent'
error: No geocoding result found for 'Sint-Amandsplein' in Gent
